#BlazingSQL and Apache Spark on Google Colab
##Analyzing Over 20M Records of Netflow Data

Below we have one of our popular workloads running with [BlazingSQL + RAPIDS AI](https://blazingdb.com) and then running the entire ETL phase again, only this time with Apache Spark + PySpark.

We think Google Colab is incredibly exciting. It solves one of the biggest painpoints developers (novice and pro alike) by giving a fully operational Python development environment right inside Google Drive!

Now you can install BlazingSQL + RAPIDS AI, the open GPU data science framework, with one install command. Query millions of rows of data for free, architect Graph, ML, AI, and so much more.

##BlazingSQL + RAPIDS AI

In [0]:
%%writefile bsql-colab.sh

#!/bin/bash


set -eu

wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/env-check.py
echo "Checking for GPU type:"
python env-check.py

if [ ! -f Miniconda3-4.5.4-Linux-x86_64.sh ]; then
    echo "Removing conflicting packages, will replace with RAPIDS compatible versions"
    # remove existing xgboost and dask installs
    pip uninstall -y xgboost dask distributed

    # intall miniconda
    echo "Installing conda"
    wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
    chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
    bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
    
    echo "Installing RAPIDS packages"
    echo "Please standby, this will take a few minutes..."
    # install RAPIDS packages
    conda install -y --prefix /usr/local \
      -c rapidsai/label/xgboost -c rapidsai -c nvidia -c conda-forge \
      python=3.6 cudatoolkit=10.0 \
      cudf=0.9.* cuml=0.9.* cugraph=0.9.* gcsfs pynvml \
      dask-cudf=0.9.* \
      rapidsai/label/xgboost::xgboost=>0.9
      
    echo "Copying shared object files to /usr/lib"
    # copy .so files to /usr/lib, where Colab's Python looks for libs
    cp /usr/local/lib/libcudf.so /usr/lib/libcudf.so
    cp /usr/local/lib/librmm.so /usr/lib/librmm.so
    cp /usr/local/lib/libxgboost.so /usr/lib/libxgboost.so
    cp /usr/local/lib/libnccl.so /usr/lib/libnccl.so
    conda install -y --prefix /usr/local -c rapidsai -c nvidia -c conda-forge -c felipeblazing/label/cuda10.0 python=3.6 cudatoolkit=10.0 blazingsql-ral blazingsql-orchestrator blazingsql-calcite blazingsql-python
    pip install flatbuffers
fi
echo ""
echo "*********************************************"
echo "Your Google Colab instance is RAPIDS ready!"
echo "*********************************************"



echo ""
echo "*********************************************"
echo "Your Google Colab instance is BlazingSQL ready!"
echo "*********************************************"


Overwriting bsql-colab.sh


In [0]:
%%time
!bash bsql-colab.sh

--2019-08-23 23:02:04--  https://github.com/rapidsai/notebooks-extended/raw/master/utils/env-check.py
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/rapidsai/notebooks-contrib/raw/master/utils/env-check.py [following]
--2019-08-23 23:02:04--  https://github.com/rapidsai/notebooks-contrib/raw/master/utils/env-check.py
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/env-check.py [following]
--2019-08-23 23:02:04--  https://raw.githubusercontent.com/rapidsai/notebooks-contrib/master/utils/env-check.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.

In [0]:
import sys, os

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

#we are working on removing the need to call these functions and just initializing them in BlazingContext
import subprocess
subprocess.Popen(['blazingsql-orchestrator', '9100', '8889', '127.0.0.1', '8890'],stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
subprocess.Popen(['java', '-jar', '/usr/local/lib/blazingsql-algebra.jar', '-p', '8890'])
import pyblazing.apiv2.context as cont
cont.runRal()

In [0]:
# Import RAPIDS AI stack
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

connection established


###Load & Query Table

In [0]:
!wget https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv

--2019-08-23 23:08:49--  https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv
Resolving blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)... 52.216.144.139
Connecting to blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)|52.216.144.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2725056295 (2.5G) [text/csv]
Saving to: ‘nf-chunk2.csv’

nf-chunk2.csv       100%[===================>]   2.54G  46.2MB/s    in 54s     

2019-08-23 23:09:43 (48.2 MB/s) - ‘nf-chunk2.csv’ saved [2725056295/2725056295]



In [0]:
%%time
#Load CSVs into GPU DataFrames (gdf)
netflow_gdf = cudf.read_csv('nf-chunk2.csv')

CPU times: user 4.18 s, sys: 1.49 s, total: 5.67 s
Wall time: 5.78 s


In [0]:
%%time
#Create BlazingSQL Tables - There is no copy in this process
bc.create_table('netflow', netflow_gdf)

CPU times: user 28.6 ms, sys: 136 µs, total: 28.8 ms
Wall time: 92.9 ms


In [0]:
%%time
sql = '''
SELECT
  a.firstSeenSrcIp as source,
  a.firstSeenDestIp as destination,
  count(a.firstSeenDestPort) as targetPorts,
  SUM(a.firstSeenSrcTotalBytes) as bytesOut,
  SUM(a.firstSeenDestTotalBytes) as bytesIn,
  SUM(a.durationSeconds) as durationSeconds,
  MIN(parsedDate) as firstFlowDate,
  MAX(parsedDate) as lastFlowDate,
  COUNT(*) as attemptCount
  FROM
  main.netflow a
  GROUP BY
  a.firstSeenSrcIp,
  a.firstSeenDestIp
  '''

result = bc.sql(sql,['netflow']).get()

# print(result.columns)

result_gdf = result.columns
edges_df = result_gdf.to_pandas()
print(edges_df.head(10))

NOTE: You no longer need to send a table list to the .sql() funtion
  firstSeenSrcIp firstSeenDestIp  ...         lastFlowDate  attemptCount
0   172.10.1.234        10.0.0.5  ...  2013-04-03 15:11:07           104
1      10.1.0.76     172.10.1.82  ...  2013-04-03 09:55:05             1
2    172.30.1.85        10.0.0.8  ...  2013-04-03 12:06:53            84
3    172.30.2.60        10.0.0.9  ...  2013-04-03 12:12:37            82
4   172.10.1.106    10.199.250.2  ...  2013-04-03 10:12:35            40
5       10.0.0.9    172.30.1.124  ...  2013-04-03 10:36:04             1
6   172.30.2.125        10.0.0.9  ...  2013-04-03 12:12:37            69
7    172.30.1.10       10.0.0.12  ...  2013-04-03 12:11:40            69
8    172.20.1.58        10.7.5.5  ...  2013-04-03 11:20:09            49
9    172.20.1.53        10.0.0.7  ...  2013-04-03 11:22:04            67

[10 rows x 9 columns]
CPU times: user 62.6 ms, sys: 10.9 ms, total: 73.5 ms
Wall time: 2.59 s


## Apache Spark

In [0]:
%%time
# Install Spark 
# Note: This installs Spark (version 2.4.1, as tested in April 2019)

!pip install pyspark

     |████████████████████████████████| 215.6MB 125kB/s 
     |████████████████████████████████| 204kB 44.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.3-py2.py3-none-any.whl size=215964963 sha256=309c5916fd287e31dd2c8d7fd305111b567fc1f6893d5d2a51e153d98c161fba
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark
CPU times: user 1.07 s, sys: 2.32 s, total: 3.39 s
Wall time: 36.3 s


In [0]:
%%time
#I copied this cell's snippet from another Google Colab by Luca Canali here: https://colab.research.google.com/github/LucaCanali/sparkMeasure/blob/master/examples/SparkMeasure_Jupyter_Colab_Example.ipynb

from pyspark.sql import SparkSession

# Create Spark Session
# This example uses a local cluster, you can modify master to use  YARN or K8S if available 
# This example downloads sparkMeasure 0.13 for scala 2_11 from maven central

spark = SparkSession \
 .builder \
 .master("local[*]") \
 .appName("PySpark Netflow Benchmark code") \
 .config("spark.jars.packages","ch.cern.sparkmeasure:spark-measure_2.11:0.13")  \
 .getOrCreate()

CPU times: user 54.6 ms, sys: 31.3 ms, total: 86 ms
Wall time: 9.41 s


###Load & Query Table

In [0]:
%%time

netflow_df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('nf-chunk2.csv')

CPU times: user 121 ms, sys: 81.4 ms, total: 202 ms
Wall time: 5min 34s


In [0]:
%%time
netflow_df.createOrReplaceTempView('netflow')

CPU times: user 822 µs, sys: 1.43 ms, total: 2.25 ms
Wall time: 200 ms


In [0]:
%%time
sql = '''
SELECT
  a.firstSeenSrcIp as source,
  a.firstSeenDestIp as destination,
  count(a.firstSeenDestPort) as targetPorts,
  SUM(a.firstSeenSrcTotalBytes) as bytesOut,
  SUM(a.firstSeenDestTotalBytes) as bytesIn,
  SUM(a.durationSeconds) as durationSeconds,
  MIN(parsedDate) as firstFlowDate,
  MAX(parsedDate) as lastFlowDate,
  COUNT(*) as attemptCount
  FROM
  netflow a
  GROUP BY
  a.firstSeenSrcIp,
  a.firstSeenDestIp
  '''

edges_df = spark.sql(sql)

edges_df.show()

+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
|      source|    destination|targetPorts|bytesOut|bytesIn|durationSeconds|      firstFlowDate|       lastFlowDate|attemptCount|
+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
| 172.10.1.13|239.255.255.250|         15|    2975|      0|              6|2013-04-03 06:36:19|2013-04-03 06:36:27|          15|
|172.30.1.204|239.255.255.250|          8|    1750|      0|              6|2013-04-03 06:36:13|2013-04-03 06:36:20|           8|
| 172.30.2.86|      172.0.0.1|          1|     540|      0|              2|2013-04-03 06:36:09|2013-04-03 06:36:09|           1|
|172.30.1.246|      172.0.0.1|         29|    2610|   2610|              0|2013-04-03 00:26:46|2013-04-03 23:06:00|          29|
| 172.30.1.51|239.255.255.250|         16|    3850|      0|             18|2013-04-03 06:35:22|20